In [15]:
import torch
import numpy as np
from IPython.display import Audio
from scipy.io.wavfile import write

In [17]:
# Load checkpoint
hifigan = torch.hub.load("bshall/hifigan:main", "hifigan_hubert_soft").cuda()
# Load mel-spectrogram
mel = torch.from_numpy(np.load("/home/yaoaksyonov/coursework/audio-diff-gan/mels/f3210686_nohash_2.spec.npy")).unsqueeze(0).cuda()
# Generate
wav, sr = hifigan.generate(mel)

Using cache found in /home/yaoaksyonov/.cache/torch/hub/bshall_hifigan_main


Removing weight norm...


In [18]:
wav.squeeze((0, 1))

tensor([ 0.0004,  0.0003,  0.0003,  ..., -0.0002, -0.0002, -0.0001],
       device='cuda:0')

In [19]:
Audio(wav.squeeze((0, 1)).cpu(), rate=sr)

In [20]:
write("sample.wav", sr, wav.squeeze((0, 1)).cpu())

AttributeError: 'torch.dtype' object has no attribute 'kind'